In [1]:
from pyvi.pyvi import ViTokenizer
import settings
from sklearn.svm import LinearSVC
import os
from gensim import corpora, matutils
from sklearn.metrics import classification_report

data_train=[]
label_train=[]
stop_words=[]
dict_words=[]
dictionary=None
# load dữ liệu
def load_data(filename):
    data=[]
    label=[]
    f = open(filename)
    next(f)
    for row in f:
        split_rows = row.split( )
#         print(split_rows)
        label.append(split_rows[0])

        for split_row in stop_words:
            if split_row in split_rows:
                split_rows.remove(split_row)
        sentences=ViTokenizer.tokenize(" ".join(str(x) for x in split_rows[1:]))
        sentences=sentences.strip('0123456789%@$.,=+-!;/()*"&^:#|\n\t\'?')
        sentences=sentences.replace("<", "")
        sentences=sentences.replace(">", "")
        data.append(sentences)
    return data,label

# loại bỏ các từ dừng
def get_stop_word():
    f = open("stopwords.txt")
    for row in f:
        stop_words.append(row.strip('0123456789%@$.,=+-!;/()*"&^:#|\n\t\''))
get_stop_word()
print(stop_words)

# xây dựng từ điển
def dict_build():
    filePath="dict.txt"
    print ('Building dictionary')
    dict_words = []
    i = 0
    for text in data_train:
        i += 1
#         print ("Step {} / {}".format(i, len(data_train)))
        words=text.split( )
#         print(words)
        dict_words.append(words)
#     print(dict_words)
    dictionary = corpora.Dictionary(dict_words)
    dictionary.filter_extremes(no_below=1, no_above=0.3)
    dictionary.save_as_text(filePath)

# load từ điển
def __load_dictionary():
    filePath="dict.txt"
    dictionary = corpora.Dictionary.load_from_text(filePath)
    return dictionary
# xây dựng vector bag of word
# vector hóa dữ liệu
def get_dense(words):
    dictionary=__load_dictionary()
    vec = dictionary.doc2bow(words)
    dense = list(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0])
    return dense
# phân loại sử dụng svm


['hả', 'bot', 'ơi', 'bạn', 'bài', 'do', 'à']


In [2]:
get_dense(['xin', 'chào', 'bot'])

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [7]:
data_train,label_train=load_data("train2.txt")
dict_build()

Building dictionary


In [8]:
data_train,label_train=load_data("train2.txt")
data_test,label_test=load_data("test2.txt")
# print(data_train)
vector_train=[]
vector_test=[]
for data_ in data_train:
#     print(data_.split( ))
    vector_train.append(get_dense(data_.split( )))
for data__ in data_test:
#     print(data_.split( ))
    vector_test.append(get_dense(data__.split( )))
print(len(vector_train))
print(len(vector_test))

142
34


In [9]:
estimator = LinearSVC(random_state=0)
print(estimator)
estimator.fit(vector_train,label_train)
y_true, y_pred = label_test, estimator.predict(vector_test)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
import pickle
v1=get_dense('giá chung_cư quận Thanh_Xuân bao_nhiêu'.split( ))
mtrix=[v1]
result=estimator.predict(mtrix)
print(result)
f = open('../model/intent_model.pkl', 'wb')
pickle.dump(estimator, f)
f.close()

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)
                   precision    recall  f1-score   support

__label__greeting       0.93      1.00      0.96        13
   __label__house       0.82      0.90      0.86        10
   __label__price       1.00      0.82      0.90        11

      avg / total       0.92      0.91      0.91        34

['__label__price']


In [6]:
data_train,label_train=load_data("train.txt")
print (data[:10])
print (label[:10])

NameError: name 'data' is not defined

In [7]:
remove_stop_word()
print(stop_words)

NameError: name 'remove_stop_word' is not defined